### Title:
__Exact simulation of the 3/2 model__

### Authors:
* Jan Baldeaux

### Abstract:
This paper discusses the exact simulation of the stock price process underlying the 3/2 model. Using a result derived by Craddock and Lennox using Lie Symmetry Analysis, we adapt the Broadie-Kaya algorithm for the simulation of affine processes to the 3/2 model. We also discuss variance reduction techniques and find that conditional Monte Carlo techniques combined with quasi-Monte Carlo point sets result in significant variance reductions.

### Link:
* [Journal](https://www.worldscientific.com/doi/abs/10.1142/S021902491250032X)
* [arXiv](http://arxiv.org/abs/1105.3297)

### References
* Baldeaux J (2012) Exact simulation of the 3/2 model. Int J Theor Appl Finan 15:1250032. https://doi.org/10.1142/S021902491250032X
* Kouarfate IR, Kouritzin MA, MacKay A (2021) Explicit Solution Simulation Method for the 3/2 Model. In: Hernández‐Hernández D, Leonardi F, Mena RH, Pardo Millán JC (eds) Advances in Probability and Mathematical Statistics. Springer International Publishing, Cham, pp 123–145

In [227]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [373]:
import numpy as np
import time
import scipy.stats as spst
import scipy.special as spsp
import copy 
#import mp math as mp

import sys
sys.path.insert(sys.path.index('')+1, 'C:/Users/27261/Desktop/3_Courses in PHBS/3_09_AppliedStochasticProcess/Project_sv32_EMC')
import pyfeng as pf
import pyfeng.ex as pfex
np.set_printoptions(precision=4)

Run only one below

## Case I: Eq. (4.2) in Baldeaux (2012)

In [374]:
mr, theta, vov, rho, sigma, intr = 2.0, 1.5, 0.2, -0.5, 1.0, 0.05
strike, spot, texp = 1, 1, 1
p_exact = 0.443059

## Case II: Set 2 in Kouarfate et al. (2021)

In [403]:
sigma, mr, theta, vov, rho = 0.06, 22.84, 0.218, 8.56, -0.99
intr = 0

strike, spot, texp = np.array([95, 100, 105]), 100, 0.2
p_exact = np.array([10.364, 7.386, 4.938])
iv_exact = pf.Bsm(1).impvol(p_exact, strike, spot, texp)
iv_exact * 100

array([43.7918, 41.4578, 39.2219])

## Case III:in Kouarfate et al. (2021)

In [392]:
sigma, mr, theta, vov, rho = 0.06, 20.48, 0.218, 3.20, -0.99
intr = 0

strike, spot, texp = np.array([95, 100, 105]), 100, 0.5
p_exact = np.array([11.724, 8.999, 6.710])
iv_exact = pf.Bsm(1).impvol(p_exact, strike, spot, texp)
iv_exact * 100

array([32.8172, 31.9686, 31.1591])

## Pricing with Time Discreteization using Euler/Milstein scheme

In [404]:
m = pfex.Sv32McTimeStep(sigma, vov, rho, mr, theta, intr)
m.set_num_params(n_path=1.6e5, dt=1/50, rn_seed=123456)
m.scheme = 2  # Exact Stepping with 1 / NCX2
m.correct_fwd = False

In [405]:
%%time
bias = m.price(strike, spot, texp) - p_exact
print(bias)

[-2.6442 -2.83   -2.6804]
CPU times: total: 125 ms
Wall time: 124 ms


## Pricing with Exact Simulation

In [406]:
m1 = pfex.Sv32McBaldeaux2012Exact(sigma, vov, rho, mr, theta, intr)
m1.set_num_params(n_path=10000, rn_seed=123456, dt=None)
m1.correct_fwd = False

In [407]:
%%time
bias = m1.price(strike, spot, texp) - p_exact
print(bias)  #-0.051878003020602914  -0.051878003020602914

[-2.7017 -2.8781 -2.7213]
CPU times: total: 1.28 s
Wall time: 1.28 s


## Pricing with IG approximation (Almost Exact Simulation)

In [408]:
m2 = pfex.Sv32McChoiKwok2023Ig(sigma, vov, rho, mr, theta, intr)
m2.set_num_params(n_path=100000, rn_seed=123456, dt=None)
m2.correct_fwd = False

In [409]:
%%time
bias = m2.price(strike, spot, texp) - p_exact
print(bias)

[-2.6202 -2.809  -2.6671]
CPU times: total: 297 ms
Wall time: 297 ms


# Pricing with FFT method

In [410]:
m3 = pf.sv_fft.Sv32Fft(sigma, vov, rho, mr, theta, intr)
m3.correct_fwd = False

In [411]:
%%time
bias = m3.price(strike, spot, texp) - p_exact
print(bias)

[-2.6437 -2.8277 -2.6754]
CPU times: total: 46.9 ms
Wall time: 42.8 ms
